# Scripts para baixar dados do TSE, descompactar e converter CSV para Parquet

### Boletins de Urna

In [ ]:
import os
import requests
import zipfile
import pandas as pd
from bs4 import BeautifulSoup

# URL principal do conjunto de dados de 2022
BASE_URL = "https://dadosabertos.tse.jus.br/dataset/resultados-2022-boletim-de-urna"

# Diretório onde os arquivos serão baixados e extraídos
BASE_DIR = "datasets"
DOWNLOAD_DIR = os.path.join(BASE_DIR, "boletins_urna_2022")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def get_zip_links():
    """Função para obter todos os links para os arquivos ZIP do primeiro e segundo turno."""
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontra todos os links que apontam para arquivos ZIP
    zip_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.endswith(".zip"):
            zip_links.append(href)
    
    return zip_links

def download_file(url, dest_folder):
    """Função para baixar um arquivo de uma URL."""
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

def extract_zip(zip_path, extract_to):
    """Função para descompactar arquivos ZIP."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def convert_csv_to_parquet(csv_path):
    """Função para converter CSV em Parquet."""
    try:
        # Lê o CSV em um DataFrame do pandas
        df = pd.read_csv(csv_path, sep=';', encoding='latin1', low_memory=False)
        # Define o nome do arquivo Parquet
        parquet_path = csv_path.replace(".csv", ".parquet")
        # Converte para o formato Parquet
        df.to_parquet(parquet_path, index=False)
        print(f"Arquivo convertido para Parquet: {parquet_path}")
        return parquet_path
    except Exception as e:
        print(f"Erro ao converter {csv_path} para Parquet: {e}")
        return None

def remove_file(file_path):
    """Função para remover arquivos (ZIP ou CSV) após a extração/conversão."""
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Arquivo removido: {file_path}")
    else:
        print(f"Arquivo não encontrado: {file_path}")

if __name__ == "__main__":
    # Passo 1: Obter os links dos arquivos ZIP
    print("Obtendo os links dos arquivos ZIP...")
    zip_links = get_zip_links()
    
    # Passo 2: Baixar e descompactar os arquivos ZIP
    for zip_url in zip_links:
        print(f"Baixando o arquivo: {zip_url}")
        zip_file = download_file(zip_url, DOWNLOAD_DIR)
        print(f"Extraindo o arquivo: {zip_file}")
        extract_zip(zip_file, DOWNLOAD_DIR)
        
        # Passo 3: Remover o arquivo ZIP após a extração
        remove_file(zip_file)

        # Passo 4: Converter CSVs para Parquet e remover CSVs
        for root, dirs, files in os.walk(DOWNLOAD_DIR):
            for file in files:
                if file.endswith(".csv"):
                    csv_path = os.path.join(root, file)
                    print(f"Convertendo CSV para Parquet: {csv_path}")
                    parquet_file = convert_csv_to_parquet(csv_path)
                    
                    if parquet_file:
                        # Remover o CSV após a conversão
                        remove_file(csv_path)

    print("Processo completo: todos os arquivos CSV foram convertidos para Parquet e removidos.")


### Candidatos

In [ ]:
import os
import requests
import zipfile
import pandas as pd

# URL do arquivo ZIP de consulta de candidatos
URL = "https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2022.zip"

# Diretório onde o arquivo será baixado e extraído
BASE_DIR = "datasets"
DOWNLOAD_DIR = os.path.join(BASE_DIR, "consulta_cand_2022")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_file(url, dest_folder):
    """Função para baixar o arquivo de uma URL."""
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

def extract_zip(zip_path, extract_to):
    """Função para descompactar arquivos ZIP."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def convert_csv_to_parquet(csv_path):
    """Função para converter CSV em Parquet."""
    try:
        df = pd.read_csv(csv_path, sep=';', encoding='latin1', low_memory=False)
        parquet_path = csv_path.replace(".csv", ".parquet")
        df.to_parquet(parquet_path, index=False)
        print(f"Arquivo convertido para Parquet: {parquet_path}")
        return parquet_path
    except Exception as e:
        print(f"Erro ao converter {csv_path} para Parquet: {e}")
        return None

def remove_file(file_path):
    """Função para remover arquivos (ZIP ou CSV) após a extração/conversão."""
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Arquivo removido: {file_path}")
    else:
        print(f"Arquivo não encontrado: {file_path}")

if __name__ == "__main__":
    # Passo 1: Baixar o arquivo ZIP
    print(f"Baixando o arquivo: {URL}")
    zip_file = download_file(URL, DOWNLOAD_DIR)
    
    # Passo 2: Extrair o conteúdo do arquivo ZIP
    print(f"Extraindo o arquivo: {zip_file}")
    extract_zip(zip_file, DOWNLOAD_DIR)
    
    # Passo 3: Remover o arquivo ZIP após a extração
    remove_file(zip_file)
    
    # Passo 4: Converter CSVs para Parquet e remover CSVs
    for root, dirs, files in os.walk(DOWNLOAD_DIR):
        for file in files:
            if file.endswith(".csv"):
                csv_path = os.path.join(root, file)
                print(f"Convertendo CSV para Parquet: {csv_path}")
                parquet_file = convert_csv_to_parquet(csv_path)
                
                if parquet_file:
                    # Remover o CSV após a conversão
                    remove_file(csv_path)

    print("Processo completo: todos os arquivos CSV foram convertidos para Parquet e removidos.")


### Prestação de Contas

In [ ]:
import os
import requests
import zipfile
import pandas as pd

# URL do arquivo ZIP de prestação de contas eleitorais
URL = "https://cdn.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_de_contas_eleitorais_candidatos_2022.zip"

# Diretório onde o arquivo será baixado e extraído
BASE_DIR = "datasets"
DOWNLOAD_DIR = os.path.join(BASE_DIR, "prestacao_contas_2022")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_file(url, dest_folder):
    """Função para baixar o arquivo de uma URL."""
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_filename

def extract_zip(zip_path, extract_to):
    """Função para descompactar arquivos ZIP."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def convert_csv_to_parquet(csv_path):
    """Função para converter CSV em Parquet."""
    try:
        df = pd.read_csv(csv_path, sep=';', encoding='latin1', low_memory=False)
        parquet_path = csv_path.replace(".csv", ".parquet")
        df.to_parquet(parquet_path, index=False)
        print(f"Arquivo convertido para Parquet: {parquet_path}")
        return parquet_path
    except Exception as e:
        print(f"Erro ao converter {csv_path} para Parquet: {e}")
        return None

def remove_file(file_path):
    """Função para remover arquivos (ZIP ou CSV) após a extração/conversão."""
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Arquivo removido: {file_path}")
    else:
        print(f"Arquivo não encontrado: {file_path}")

if __name__ == "__main__":
    # Passo 1: Baixar o arquivo ZIP
    print(f"Baixando o arquivo: {URL}")
    zip_file = download_file(URL, DOWNLOAD_DIR)
    
    # Passo 2: Extrair o conteúdo do arquivo ZIP
    print(f"Extraindo o arquivo: {zip_file}")
    extract_zip(zip_file, DOWNLOAD_DIR)
    
    # Passo 3: Remover o arquivo ZIP após a extração
    remove_file(zip_file)
    
    # Passo 4: Converter CSVs para Parquet e remover CSVs
    for root, dirs, files in os.walk(DOWNLOAD_DIR):
        for file in files:
            if file.endswith(".csv"):
                csv_path = os.path.join(root, file)
                print(f"Convertendo CSV para Parquet: {csv_path}")
                parquet_file = convert_csv_to_parquet(csv_path)
                
                if parquet_file:
                    # Remover o CSV após a conversão
                    remove_file(csv_path)

    print("Processo completo: todos os arquivos CSV foram convertidos para Parquet e removidos.")
